In [1]:
import pandas as pd
import os
import numpy as np
DATA_PATH = '../../Dataset/'
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
def porc_na(df):
    df_na = pd.DataFrame({'Variable'      : df.columns.values,
                          'Missing (%)'     : np.round(100 * df.isna().sum().values / df.shape[0] ,3)})
    return df_na

In [2]:
%%time
food_dataset = pd.read_csv(os.path.join('','en.openfoodfacts.org.products.tsv'), encoding='utf-8', sep='\t').reset_index(drop = True)
food_dataset = food_dataset[(food_dataset['nutrition-score-fr_100g'].isna()==0)].reset_index(drop = True)
df_train     = pd.read_csv(os.path.join(DATA_PATH,'food_train.tsv'), index_col='Index', encoding='utf-8', sep='\t').reset_index(drop = True)
df_test      = pd.read_csv(os.path.join(DATA_PATH,'food_X_test.tsv'), index_col='Index', encoding='utf-8', sep='\t').reset_index(drop = True)
print(food_dataset.shape)
print(df_train.shape)
print(df_test.shape)
print(df_train.shape[0]+df_test.shape[0] )


/media/soel/INVESTIGACION/DATATHONES/env_datathon/lib/python3.6/site-packages/decorator.py:232: DtypeWarning: Columns (0,3,5,19,20,24,25,26,27,28,36,37,38,39,48) have mixed types.Specify dtype option on import or set low_memory=False.
  return caller(func, *(extras + args), **kw)


(254856, 163)
(102028, 58)
(25508, 57)
127536
CPU times: user 10.5 s, sys: 798 ms, total: 11.3 s
Wall time: 11.4 s


In [3]:
df_test.columns

Index(['creator', 'created_t', 'created_datetime', 'last_modified_t',
       'last_modified_datetime', 'product_name', 'generic_name', 'quantity',
       'packaging', 'packaging_tags', 'brands', 'brands_tags', 'categories',
       'categories_tags', 'categories_en', 'origins', 'origins_tags',
       'manufacturing_places', 'manufacturing_places_tags', 'labels',
       'labels_tags', 'labels_en', 'emb_codes', 'emb_codes_tags',
       'first_packaging_code_geo', 'cities', 'cities_tags', 'purchase_places',
       'stores', 'countries', 'countries_tags', 'countries_en',
       'ingredients_text', 'allergens', 'allergens_en', 'traces',
       'traces_tags', 'traces_en', 'serving_size', 'no_nutriments',
       'additives_n', 'additives', 'additives_tags', 'additives_en',
       'ingredients_from_palm_oil_n', 'ingredients_from_palm_oil',
       'ingredients_from_palm_oil_tags',
       'ingredients_that_may_be_from_palm_oil_n',
       'ingredients_that_may_be_from_palm_oil',
       'ingredient

In [4]:
variables = ['creator', 'created_t', 'created_datetime', 'last_modified_t',
       'last_modified_datetime', 'product_name', 'generic_name', 'quantity',
       'packaging', 'packaging_tags', 'brands', 'brands_tags', 'categories',
       'categories_tags', 'categories_en', 'origins', 'origins_tags',
       'manufacturing_places', 'manufacturing_places_tags', 'labels',
       'labels_tags', 'labels_en', 'emb_codes', 'emb_codes_tags',
       'first_packaging_code_geo', 'cities', 'cities_tags', 'purchase_places',
       'stores', 'countries', 'countries_tags', 'countries_en',
       'ingredients_text', 'allergens', 'allergens_en', 'traces',
       'traces_tags', 'traces_en', 'serving_size', 'no_nutriments',
       'additives_n', 'additives', 'additives_tags', 'additives_en',
       'ingredients_from_palm_oil_n', 'ingredients_from_palm_oil',
       'ingredients_from_palm_oil_tags',
       'ingredients_that_may_be_from_palm_oil_n',
       'ingredients_that_may_be_from_palm_oil',
       'ingredients_that_may_be_from_palm_oil_tags', 'pnns_groups_1',
       'pnns_groups_2', 'states', 'states_tags', 'states_en', 'main_category',
       'main_category_en']

In [5]:
def generate_unique(df):
    suma = '_'
    for i in variables:
        suma = suma + df[i].astype(str).fillna('VACIO')
    df['unique_col'] = suma

In [7]:
generate_unique(food_dataset)
generate_unique(df_train)
generate_unique(df_test)

In [8]:
%%time
df_train_new = df_train.merge(food_dataset[['nutrition-score-fr_100g','unique_col']],how = 'left',on = ['unique_col']) 
df_test_new  = df_train_new.drop_duplicates()
df_test_new  = df_test.merge(food_dataset[['nutrition-score-fr_100g','unique_col']],how = 'left',on = ['unique_col']) 
df_test_new  = df_test_new.drop_duplicates()

CPU times: user 3.32 s, sys: 3.35 ms, total: 3.33 s
Wall time: 3.33 s


In [9]:
df_test_new

,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,main_category,main_category_en,unique_col,nutrition-score-fr_100g
0,openfoodfacts-contributors,1408810605,2014-08-23T16:16:45Z,1468740504,2016-07-17T07:28:24Z,Tomato Ketchup,Tomaten Ketchup,"500 ml, 580 g","Flasche,Kunststoff,flasche","flasche,kunststoff,flasche","McDonald's,Develey","mcdonald-s,develey","Groceries,Sauces,Tomato sauces,Ketchup","en:groceries,en:sauces,en:tomato-sauces,en:ket...","Groceries,Sauces,Tomato sauces,Ketchup",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deutschland,en:germany,Germany,"Tomatenmark* (66%), Glukose-Fruktose-Sirup, Br...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,[ tomatenmark -> de:tomatenmark ] [ 66 -> d...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,Fat and sauces,Dressings and sauces,"en:to-be-completed, en:nutrition-facts-complet...","en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",en:groceries,Groceries,_openfoodfacts-contributors14088106052014-08-2...,14.0
1,usda-ndb-import,1489066307,2017-03-09T13:31:47Z,1489066307,2017-03-09T13:31:47Z,Grated Italian Style Blend Cheese,NaN,NaN,NaN,NaN,Western Family,western-family,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,en:united-states,United States,"Parmesan cheese (cultured part-skim milk, salt...",NaN,NaN,NaN,NaN,NaN,5 g (2 tsp),NaN,0.0,[ parmesan-cheese -> en:parmesan-cheese ] [...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,"en:to-be-completed, en:nutrition-facts-complet...","en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",NaN,NaN,_usda-ndb-import14890663072017-03-09T13:31:47Z...,23.0
2,usda-ndb-import,1489137369,2017-03-10T09:16:09Z,1489137370,2017-03-10T09:16:10Z,Empanadas De Pollo Chicken Turnovers,NaN,NaN,NaN,NaN,Quirch,quirch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,en:united-states,United States,"Filling ingredients: chicken, water, cornstarc...",NaN,NaN,NaN,NaN,NaN,108 g (1 TURNOVER),NaN,3.0,[ filling-ingredients -> en:filling-ingredien...,"en:e330,en:e509,en:e621","E330 - Citric acid,E509 - Calcium chloride,E62...",0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,"en:to-be-completed, en:nutrition-facts-complet...","en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",NaN,NaN,_usda-ndb-import14891373692017-03-10T09:16:09Z...,8.0
3,usda-ndb-import,1489090619,2017-03-09T20:16:59Z,1489090619,2017-03-09T20:16:59Z,Ice Cream,NaN,NaN,NaN,NaN,Topco Associates Inc.,topco-associates-inc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,en:united-states,United States,"Milkfat and nonfat milk, sugar, high fructose ...",NaN,NaN,NaN,NaN,NaN,65 g (0.5 cup),NaN,3.0,[ milkfat-and-nonfat-milk -> en:milkfat-and-n...,"en:e412,en:e466,en:e471","E412 - Guar gum,E466 - Sodium carboxy methyl c...",0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,"en:to-be-completed, en:nutrition-facts-complet...","en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",NaN,NaN,_usda-ndb-import14890906192017-03-09T20:16:59Z...,12.0
4,usda-ndb-import,1489077105,2017-03-09T16:31:45Z,1489077105,2017-03-09T16:31:45Z,"Organic Dre